# Bibliotecas

In [68]:
import pandas as pd
import requests
import json
import os
from datetime import datetime
import time
import aiohttp # OBRIGATÓRIO: Precisa instalar o pacote, segue o comando da instalação: pip install aiohttp
import asyncio # OBRIGATÓRIO: Precisa instalar o pacote, segue o comando da instalação: pip install asyncio
import glob
import re
import numpy as np
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.float_format', str)

# Início Log

In [69]:
main_start = time.time()

## Parâmetros

In [70]:
data_ingestao = datetime.today().strftime('%Y%m%d')

diretorio_bronze = 'dados/bronze'
bronze_localidades =  diretorio_bronze + '/localidades'
bronze_idh = diretorio_bronze +'/idh'
bronze_populacao = diretorio_bronze +'/populacao'
bronze_pib = diretorio_bronze +'/pib'

diretorio_prata = 'dados/prata'
prata_localidades =  diretorio_prata + '/localidades'
prata_idh = diretorio_prata +'/idh'
prata_populacao = diretorio_prata +'/populacao'
prata_pib = diretorio_prata +'/pib'

diretorio_ouro = 'dados/ouro'

# Camada Bronze

### Funções

In [71]:
async def SalvarIbgeJson(diretorio, nome, dados):
    try:
    
        if not os.path.exists(diretorio):
            try:
                os.makedirs(diretorio)
            except Exception as e:
                raise
        with open(os.path.join(diretorio, nome+'.json'), 'w') as f:
            json.dump(dados, f, ensure_ascii=False, indent=4)
    
        return 'Json salvo'
        
    except:
        return 'Json não salvo: ' + e  

In [72]:
# Consulta a api do IBGE
async def ConsultaIbgeJson(url):
    try:
        response = requests.get(url)
        return response.json()
    except Exception as e:
        return 'Api erro: '+ e  

## Localidades

### Estado

In [73]:
# Realizando  consulta no IBGE os dados dos estados brasileiros 
url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados?orderBy=nome"
ibge_estados = await ConsultaIbgeJson(url)

# Salvar os dados dos estados na camada bronze
await SalvarIbgeJson(bronze_localidades,data_ingestao+'_estados',ibge_estados)

'Json salvo'

In [74]:
df_estados = pd.json_normalize(ibge_estados)
df_estados = df_estados[['id','sigla']]

### Municípios

In [75]:
async def SalvarLocalidadesEstados(respJson):
    uf = pd.json_normalize(respJson[0])
    uf = uf['microrregiao.mesorregiao.UF.sigla'][0]
    await SalvarIbgeJson(bronze_localidades+'/municipios',data_ingestao + '_'+ uf,respJson)  

In [76]:
async def GetLocalidadesEstados(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
            await SalvarLocalidadesEstados(resp)
    except Exception as e:
        print("Erro de consultar a url {} erro para {}.".format(url, e.__class__))

In [77]:
async def MainLocalidadesEstados(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[GetLocalidadesEstados(url, session) for url in urls])  

In [78]:
# Realizando consulta no IBGE os dados dos munícipios de cada estado do brasil  
urls = []
for index, estado in df_estados.iterrows():
    url = f"https://servicodados.ibge.gov.br/api/v1/localidades/estados/"+ str(estado['id'])+"/municipios"
    urls.append(url)    

start = time.time()
await MainLocalidadesEstados(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 3.5157227516174316 segundos para consultar as 27 urls.


## População

### Cor e Raça

In [79]:
async def SalvarPopulacaoCorRaca(respJson):
    dfPopulacaoCorRaca = pd.json_normalize(respJson)   
    municipio = dfPopulacaoCorRaca[dfPopulacaoCorRaca['NC'] == '6']['D1C'].iloc[0]
    uf = dfPopulacaoCorRaca[dfPopulacaoCorRaca['NC'] == '3']['D1C'].iloc[0]
    uf = df_estados[df_estados['id'] == int(uf)]['sigla'].iloc[0]
    
    ret = await SalvarIbgeJson(bronze_populacao + '/estados/'+ uf + '/municipios/' + municipio \
            ,data_ingestao + '_populacao_cor_raca_'+str(municipio),respJson) 
    return ret

In [80]:
async def GetPopulacaoCorRaca(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
        return await SalvarPopulacaoCorRaca(resp)
    except Exception as e:
        print("Erro de consultar a url {} erro para {}.".format(url, e.__class__))

In [81]:
async def MainPopulacaoCorRaca(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[GetPopulacaoCorRaca(url, session) for url in urls])
    return ret

In [82]:
urls = []
for index, estado in df_estados.iterrows():
    
    diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + estado['sigla'] + '.json'
    with open(diretorio_estado) as f:
        municipios = json.load(f)
        municipios = pd.json_normalize(municipios)
        municipios = municipios['id']

        for municipio in municipios:
            url = f"https://apisidra.ibge.gov.br/values/t/9605/n1/1/n3/"+ str(estado['id']) +"/n6/" + str(municipio) +"/v/93/p/all/c86/all"
            urls.append(url)

start = time.time()
ret = await MainPopulacaoCorRaca(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 54.153632164001465 segundos para consultar as 5570 urls.


### Sexo e Range de Idade

In [83]:
async def SalvarPopulacaoSexoRangeIdade(respJson):
    dfPopulacaoSexoRangeIdade = pd.json_normalize(respJson)   
    municipio = dfPopulacaoSexoRangeIdade[dfPopulacaoSexoRangeIdade['NC'] == '6']['D1C'].iloc[0]
    uf = dfPopulacaoSexoRangeIdade[dfPopulacaoSexoRangeIdade['NC'] == '3']['D1C'].iloc[0]
    uf = df_estados[df_estados['id'] == int(uf)]['sigla'].iloc[0]
    
    ret = await SalvarIbgeJson(bronze_populacao + '/estados/'+ uf + '/municipios/' + municipio \
            ,data_ingestao + '_populacao_sexo_idade_'+str(municipio),respJson) 
    return ret

In [84]:
async def GetPopulacaoSexoRangeIdade(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
        return await SalvarPopulacaoSexoRangeIdade(resp)
    except Exception as e:
        print("Erro de consultar a url {} erro para {}.".format(url, e.__class__))

In [85]:
async def MainPopulacaoSexoRangeIdade(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[GetPopulacaoSexoRangeIdade(url, session) for url in urls])
    return ret

In [86]:
urls = []
for index, estado in df_estados.iterrows():
    
    diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + estado['sigla'] + '.json'
    with open(diretorio_estado) as f:
        municipios = json.load(f)
        municipios = pd.json_normalize(municipios)
        municipios = municipios['id']

        for municipio in municipios:
            url = f"https://apisidra.ibge.gov.br/values/t/9514/n1/1/n3/"+str(estado['id'])+"/n6/"+str(municipio)+"/v/93/p/all/c2/all/c287/93070,93084,93085,93086,93087,93088,93089,93090,93091,93092,93093,93094,93095,93096,93097,93098,49108,49109,60040,60041,6653"
            urls.append(url)

start = time.time()
ret = await MainPopulacaoSexoRangeIdade(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 84.97164392471313 segundos para consultar as 5570 urls.


## PIB

In [87]:
async def SalvarPib(respJson):
    dfPib = pd.json_normalize(respJson)   
    municipio = dfPib[dfPib['NC'] == '6']['D1C'].iloc[0]
    uf = dfPib[dfPib['NC'] == '3']['D1C'].iloc[0]
    uf = df_estados[df_estados['id'] == int(uf)]['sigla'].iloc[0]
    
    ret = await SalvarIbgeJson(bronze_pib + '/estados/'+ uf + '/municipios/' + municipio \
            ,data_ingestao + '_pib_'+str(municipio),respJson) 
    return ret

In [88]:
async def GetPib(url, session):
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
        return await SalvarPib(resp)
    except Exception as e:
        print("Erro de consultar a url {} erro para {}.".format(url, e.__class__))

In [89]:
async def MainPib(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[GetPib(url, session) for url in urls])
    return ret

In [90]:
urls = []
for index, estado in df_estados.iterrows():
    
    diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + estado['sigla'] + '.json'
    with open(diretorio_estado) as f:
        municipios = json.load(f)
        municipios = pd.json_normalize(municipios)
        municipios = municipios['id']

        for municipio in municipios:
              url = f"https://apisidra.ibge.gov.br/values/t/5938/n1/1/n3/"+str(estado['id'])+"/n6/"+str(municipio)+"/v/37/p/all"
              urls.append(url)           

start = time.time()
ret = await MainPib(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls))) 

Tempo de consulta 35.62893199920654 segundos para consultar as 5570 urls.


## IDH

### Estados

In [91]:
for index, estado in df_estados.iterrows():
     url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(estado['id'])
     idh = await ConsultaIbgeJson(url)

     await SalvarIbgeJson(bronze_idh + '/estados', data_ingestao + '_idh_'+ estado['sigla'],idh)    

### Munícipios (IDHM) 

#### Funções

In [92]:
async def SalvarIhdm(respJson,municipio):
    
    uf = await ConsultaIbgeJson('https://servicodados.ibge.gov.br/api/v1/localidades/municipios/'+ municipio)
    uf = str(pd.json_normalize(uf)['microrregiao.mesorregiao.UF.sigla'][0])
    respJson[0]['res'][0]['localidade'] = municipio

    ret = await SalvarIbgeJson(bronze_idh + '/idhm/'+ uf + '/municipios' ,data_ingestao + '_idhm_'+ municipio,respJson) 
    return ret

In [93]:
async def GetIhdm(url, session):
    error = True
    try:
        async with session.get(url=url) as response:
            resp = await response.json()
        await SalvarIhdm(resp,url.rsplit('/',1)[1])
        error = False
    except Exception as e:
         error = True
    finally:
        return error

In [94]:
async def GetIhdmLoop(url, session):
    error = False
    error = await GetIhdm(url, session)
    while error:
        error = await GetIhdm(url, session)        

In [95]:
async def MainGetIhdm(urls):
    async with aiohttp.ClientSession() as session:
        ret = await asyncio.gather(*[GetIhdmLoop(url, session) for url in urls])
    return ret

#### AC

In [99]:
urls = []
uf = 'AC'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 20.823083877563477 segundos para consultar as 22 urls.


#### AL

In [98]:
urls = []
uf = 'AL'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 101.2027199268341 segundos para consultar as 102 urls.


#### AP

In [100]:
urls = []
uf = 'AP'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 15.59290099143982 segundos para consultar as 16 urls.


#### AM

In [101]:
urls = []
uf = 'AM'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 59.77863311767578 segundos para consultar as 62 urls.


#### BA

In [102]:
urls = []
uf = 'BA'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 391.6915693283081 segundos para consultar as 417 urls.


#### CE

In [103]:
urls = []
uf = 'CE'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 173.0133490562439 segundos para consultar as 184 urls.


#### DF

In [104]:
urls = []
uf = 'DF'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 1.0526931285858154 segundos para consultar as 1 urls.


#### ES

In [105]:
urls = []
uf = 'ES'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 71.43046140670776 segundos para consultar as 78 urls.


#### GO

In [106]:
urls = []
uf = 'GO'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 232.82219552993774 segundos para consultar as 246 urls.


#### MA

In [107]:
urls = []
uf = 'MA'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 206.0578007698059 segundos para consultar as 217 urls.


#### MT

In [108]:
urls = []
uf = 'MT'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 135.98740148544312 segundos para consultar as 141 urls.


#### MS

In [109]:
urls = []
uf = 'MS'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 73.24209952354431 segundos para consultar as 79 urls.


#### MG

In [110]:
urls = []
uf = 'MG'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 791.961100101471 segundos para consultar as 853 urls.


#### PA

In [111]:
urls = []
uf = 'PA'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 132.65338516235352 segundos para consultar as 144 urls.


#### PB

In [112]:
urls = []
uf = 'PB'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 207.91943788528442 segundos para consultar as 223 urls.


#### PR

In [113]:
urls = []
uf = 'PR'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 373.3944172859192 segundos para consultar as 399 urls.


#### PE

In [114]:
urls = []
uf = 'PE'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 175.5269796848297 segundos para consultar as 185 urls.


#### PI

In [115]:
urls = []
uf = 'PI'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 206.64634728431702 segundos para consultar as 224 urls.


#### RJ

In [116]:
urls = []
uf = 'RJ'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 85.08200144767761 segundos para consultar as 92 urls.


#### RN

In [117]:
urls = []
uf = 'RN'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 157.87485718727112 segundos para consultar as 167 urls.


#### RS

In [118]:
urls = []
uf = 'RS'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 470.57864570617676 segundos para consultar as 497 urls.


#### RO

In [119]:
urls = []
uf = 'RO'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 47.7691445350647 segundos para consultar as 52 urls.


#### RR

In [120]:
urls = []
uf = 'RR'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 13.765439510345459 segundos para consultar as 15 urls.


#### SC

In [121]:
urls = []
uf = 'SC'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 273.1189546585083 segundos para consultar as 295 urls.


#### SP

In [122]:
urls = []
uf = 'SP'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 599.2440133094788 segundos para consultar as 645 urls.


#### SE

In [123]:
urls = []
uf = 'SE'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 69.14277577400208 segundos para consultar as 75 urls.


#### TO

In [124]:
urls = []
uf = 'TO'
diretorio_estado = bronze_localidades +'/municipios/'+ data_ingestao + '_' + uf + '.json'
with open(diretorio_estado) as f:
    municipios = json.load(f)
    municipios = pd.json_normalize(municipios)
    municipios = municipios['id']

    for municipio in municipios:
        url = f"https://servicodados.ibge.gov.br/api/v1/pesquisas/37/indicadores/30255/resultados/"+str(municipio)
        urls.append(url)

start = time.time()
ret = await MainGetIhdm(urls)
end = time.time()
print("Tempo de consulta {} segundos para consultar as {} urls.".format(end - start, len(urls)))

Tempo de consulta 128.42257976531982 segundos para consultar as 139 urls.


# Camada Prata

## Funções

In [142]:
def CriarDiretorioPrata(diretorio):
    ret = True
    try:
        if not os.path.exists(diretorio):
            try:
                os.makedirs(diretorio)
                ret = True
            except Exception as e:
                ret = False
                raise
    except:
        ret = False 
    finally:
        return ret

## Localidades

### Município

In [140]:
bronze_municipio = bronze_localidades + '/municipios' 
municipios_json = [bronze_municipio+'/'+municipio_json for municipio_json in os.listdir(bronze_municipio) \
                   if municipio_json.endswith('.json') and data_ingestao in municipio_json]

df = pd.DataFrame()
for municipio_arquivo in municipios_json:
       
    with open(municipio_arquivo) as f:
        municipio = json.load(f)
        municipio = pd.json_normalize(municipio)
        municipio['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([municipio,df])

prata_estado = prata_localidades + '/municipios.csv'

if(CriarDiretorioPrata(prata_localidades)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_estado, mode='a', encoding='utf-8', sep=';',index=False)

del df
del municipios_json

### Estado

In [141]:
bronze_estado = bronze_localidades + '/' + data_ingestao + '_estados.json'

with open(bronze_estado) as f:
    estados = json.load(f)
    estados = pd.json_normalize(estados)
    estados['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')

prata_estado = prata_localidades + '/estados.csv'

if(CriarDiretorioPrata(prata_localidades)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    estados.to_csv(prata_estado, mode='a', encoding='utf-8', sep=';',index=False)

del estados

## População

### Cor e Raça

In [143]:
bronze_populacao_cor_raca = bronze_populacao + '/estados'
populacao_cor_raca_json = glob.glob(bronze_populacao_cor_raca+'/**/*.json', recursive=True)
populacao_cor_raca_json = [json.replace('\\', '/') for json in populacao_cor_raca_json]

populacao_cor_raca_json = [bronze_populacao_cor_raca for bronze_populacao_cor_raca in populacao_cor_raca_json \
                           if 'populacao_cor_raca' in bronze_populacao_cor_raca and data_ingestao in bronze_populacao_cor_raca] 

df = pd.DataFrame()
for populacao_cor_raca_arquivo in populacao_cor_raca_json:
       
    with open(populacao_cor_raca_arquivo) as f:
        populacao_cor_raca = json.load(f)
        populacao_cor_raca = pd.json_normalize(populacao_cor_raca)
        populacao_cor_raca['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([populacao_cor_raca,df])
        
prata_populacao_cor_racas = prata_populacao + '/populacao_cor_racas.csv'

if(CriarDiretorioPrata(prata_populacao)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_populacao_cor_racas, mode='a', encoding='utf-8', sep=';',index=False)

del df
del populacao_cor_raca_json

### Sexo e Range de Idade

In [144]:
bronze_populacao_sexo_idade = bronze_populacao + '/estados'
populacao_sexo_idade_json = glob.glob(bronze_populacao_sexo_idade+'/**/*.json', recursive=True)
populacao_sexo_idade_json = [json.replace('\\', '/') for json in populacao_sexo_idade_json]

populacao_sexo_idade_json = [bronze_populacao_sexo_idade for bronze_populacao_sexo_idade in populacao_sexo_idade_json \
                           if 'populacao_sexo_idade' in bronze_populacao_sexo_idade and data_ingestao in bronze_populacao_sexo_idade] 

df = pd.DataFrame()
for populacao_sexo_idade_arquivo in populacao_sexo_idade_json:
    
    with open(populacao_sexo_idade_arquivo) as f:
        populacao_sexo_idade = json.load(f)
        populacao_sexo_idade = pd.json_normalize(populacao_sexo_idade)
        populacao_sexo_idade['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([populacao_sexo_idade,df])
       
prata_populacao_sexo_idade = prata_populacao + '/populacao_sexo_idade.csv'

if(CriarDiretorioPrata(prata_populacao)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_populacao_sexo_idade, mode='a', encoding='utf-8', sep=';',index=False)
        
del df
del populacao_sexo_idade_json

## PIB

In [ ]:
bronze_pib_bruto = bronze_pib + '/estados'
pib_json = glob.glob(bronze_pib_bruto+'/**/*.json', recursive=True)
pib_json = [json.replace('\\', '/') for json in pib_json]

pib_json = [pib_bruto for pib_bruto in pib_json if '_pib_' in pib_bruto and data_ingestao in pib_bruto]

In [146]:
bronze_pib_bruto = bronze_pib + '/estados'
pib_json = glob.glob(bronze_pib_bruto+'/**/*.json', recursive=True)
pib_json = [json.replace('\\', '/') for json in pib_json]

pib_json = [pib_bruto for pib_bruto in pib_json if '_pib_' in pib_bruto and data_ingestao in pib_bruto]

df = pd.DataFrame()
for pib_arquivo in pib_json:
       
    with open(pib_arquivo) as f:
        pib = json.load(f)
        pib = pd.json_normalize(pib)
        pib['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([pib,df])
        
prata_pib_bruto = prata_pib + '/pib.csv'

if(CriarDiretorioPrata(prata_pib)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_pib_bruto, mode='a', encoding='utf-8', sep=';',index=False)
        
del df
del pib_json

## IDH

### Estado

In [147]:
bronze_idh_estado = bronze_idh + '/estados'
idh_estado_json = glob.glob(bronze_idh_estado+'/**/*.json', recursive=True)
idh_estado_json = [json.replace('\\', '/') for json in idh_estado_json]

idh_estado_json = [idh_estado for idh_estado in idh_estado_json  if '_idh_' in idh_estado and data_ingestao in idh_estado] 

df = pd.DataFrame()
for idh_estado_arquivo in idh_estado_json:
    
    with open(idh_estado_arquivo) as f:
        idh_estado = json.load(f)
        idh_estado = pd.json_normalize(idh_estado[0]['res'])
        idh_estado['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([idh_estado,df])

prata_idh_estado = prata_idh + '/IdhEstado.csv'

if(CriarDiretorioPrata(prata_idh)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_idh_estado, mode='a', encoding='utf-8', sep=';',index=False)
        
del df
del idh_estado_json

### Município (IDHM)

In [148]:
bronze_idhm = bronze_idh + '/idhm' 
idhm_json = glob.glob(bronze_idhm+'/**/*.json', recursive=True)
idhm_json = [json.replace('\\', '/') for json in idhm_json]

idhm_json = [idhm for idhm in idhm_json if '_idhm_' in idhm and data_ingestao in idhm]

df = pd.DataFrame()
for idhm_arquivo in idhm_json:
       
    with open(idhm_arquivo) as f:
        idhm = json.load(f)
        idhm = pd.json_normalize(idhm[0]['res'])
        idhm['data_ingestao'] = datetime.strptime(data_ingestao, '%Y%m%d')
        df = pd.concat([idhm,df])

prata_idhm = prata_idh + '/IdhMunicipio.csv'

if(CriarDiretorioPrata(prata_idh)):
    # É realizado somente o append do dados da BRONZE para a PRATA
    df.to_csv(prata_idhm, mode='a', encoding='utf-8', sep=';',index=False)

del df
del prata_idhm

# Camada Ouro

### Funções

In [178]:
def CreatePkPopulacao(idMunicipio,Ano,idSexo,idRangeIdade,idCorERaca):
    idSexoNan = float(idSexo)
    idRangeIdadeNan = float(idRangeIdade)
    idCorERacaNan = float(idCorERaca)
    pk = ''
    
    if np.isnan(idSexoNan) and ~np.isnan(idRangeIdadeNan) and ~np.isnan(idCorERacaNan):
        pk = str(idMunicipio) + str(Ano) + str(idRangeIdade) + str(idCorERaca)
        
    elif ~np.isnan(idSexoNan) and np.isnan(idRangeIdadeNan) and ~np.isnan(idCorERacaNan):
        pk = str(idMunicipio) + str(Ano) + str(idSexo) + str(idSexo) + str(idCorERaca)

    elif ~np.isnan(idSexoNan) and ~np.isnan(idRangeIdadeNan) and np.isnan(idCorERacaNan):
        pk = str(idMunicipio) + str(Ano) + str(idSexo) + str(idRangeIdade)
        
    elif np.isnan(idCorERacaNan) and np.isnan(idSexoNan) and np.isnan(idCorERacaNan):
        pk = str(idMunicipio) +str(Ano)  
        
    else:
         pk = str(idMunicipio) + str(Ano) + str(idSexo) + str(idRangeIdade) + str(idCorERaca)
        
    return pk

In [150]:
def ConverterNumero(num):
    ret = 0
    
    # Verificar o campo se tem valor númerico nos formatos:  123 |.123 |1,23 
    p = r'[\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+'
    ret = re.findall(p, str(num))[0] if re.search(p, str(num)) else 0
    
    return ret

In [199]:
def CreatePkFatoIbge(PkIdMunicipio,Ano,PkPopulacao,PkPib,PkIdhMunicipio):
      
    pk = ''
    
    if ~pd.isnull(PkPopulacao) and pd.isnull(PkPib) and pd.isnull(PkIdhMunicipio):
        pk = str(PkIdMunicipio) + str(Ano) + str(PkPopulacao) 
        
    elif pd.isnull(PkPopulacao) and ~pd.isnull(PkPib) and pd.isnull(PkIdhMunicipio):
        pk = str(PkIdMunicipio) + str(Ano) + str(PkPib) 

    elif  pd.isnull(PkPopulacao) and pd.isnull(PkPib) and ~pd.isnull(PkIdhMunicipio):
        pk = str(PkIdMunicipio) + str(Ano) + str(PkIdhMunicipio)
        
    elif pd.isnull(PkPopulacao) and pd.isnull(PkPib) and pd.isnull(PkIdhMunicipio):
        pk = str(PkIdMunicipio) +str(Ano)  
        
    else:
         pk = str(PkIdMunicipio) + str(Ano) + str(PkPopulacao) + str(PkPib) + str(PkIdhMunicipio)
        
    return pk

## Localidade

In [151]:
estado = pd.read_csv(prata_localidades + '/estados.csv',sep=';')  

#Filtrando somente as colunas necessárias
estado = estado[['id','sigla','nome','regiao.nome']]
estado = estado.rename(columns={'id': "IdEstado",'sigla':'Uf','nome':'Estado','regiao.nome':'Regiao'})

municipio = pd.read_csv(prata_localidades + '/municipios.csv',sep=';')  

municipio = municipio[['id','nome','microrregiao.mesorregiao.UF.id']]
municipio = municipio.rename(columns={'id': "PkIdMunicipio",'nome':'Municipio','microrregiao.mesorregiao.UF.id':'IdEstado'})

localidade_ouro = diretorio_ouro + '/localidade.csv'
pd.merge(municipio, estado, on='IdEstado', how='inner').to_csv(localidade_ouro, encoding='utf-8', sep=';',index=False)

## População

In [160]:
cor_racas = pd.read_csv(prata_populacao + '/populacao_cor_racas.csv',sep=';')  

# ------------------------------------ Criando df população cor e raça ------------------------------------ #

#Filtro somente os municípios
cor_racas = cor_racas[cor_racas['NC'] == '6']

#Filtrando somente as colunas necessárias
cor_racas = cor_racas[['D1C','D3N','D4C','D4N','V']]

filter = cor_racas['D1C'].str.contains('Código')
cor_racas = cor_racas[~filter]
cor_racas = cor_racas.rename(columns={'D1C': "IdMunicipio",
                        'D3N':'Ano',
                        'D4C':'IdCorERaca',
                        'D4N':'CorERaca',
                        'V':'QuantidadeCorERaca'
                  })

# ------------------------------------ Criando df população sexo e range ------------------------------------ #

sexo_idade = pd.read_csv(prata_populacao + '/populacao_sexo_idade.csv',sep=';')  

#Filtro somente os municípios
sexo_idade = sexo_idade[sexo_idade['NC'] == '6']

#Filtrando somente as colunas necessárias
sexo_idade = sexo_idade[['D1C','D3N','D4C','D4N','D5C','D5N','V']]
filter = sexo_idade['D1C'].str.contains('Código')
sexo_idade = sexo_idade[~filter]
sexo_idade = sexo_idade.rename(columns={'D1C': "IdMunicipio",
                       'D3N':'Ano',
                        'D4C':'IdSexo',
                        'D4N':'Sexo',
                        'D5C':'IdRangeIdade',
                        'D5N':'RangeIdade',
                        'V':'QuantidadeSexoRangeIdade'
                  })

# ------------------------------------ Criando df população cor e raça ------------------------------------ #
populacao = pd.merge(sexo_idade, cor_racas, on=['IdMunicipio','Ano'], how='outer')

populacao['PkPopulacao'] = populacao.apply(lambda x: \
                            CreatePkPopulacao(x['IdMunicipio'], x['Ano'],x['IdSexo'],x['IdRangeIdade'],x['IdCorERaca']) \
                            ,axis=1)

# Tipando as colunas númericos
populacao['IdMunicipio'] = populacao['IdMunicipio'].astype('Int64')
populacao['Ano'] = populacao['Ano'].astype('Int64')
populacao['IdSexo'] = populacao['IdSexo'].astype('Int64')
populacao['IdRangeIdade'] = populacao['IdRangeIdade'].astype('Int64')
populacao['QuantidadeSexoRangeIdade'] = populacao['QuantidadeSexoRangeIdade'].apply(lambda x: ConverterNumero(x))
populacao['QuantidadeCorERaca'] = populacao['QuantidadeCorERaca'].apply(lambda x: ConverterNumero(x))
populacao['IdCorERaca'] = populacao['IdCorERaca'].astype('Int64')

populacao_ouro = diretorio_ouro + '/populacao.csv'
populacao.to_csv(populacao_ouro, encoding='utf-8', sep=';',index=False)

del populacao, sexo_idade, cor_racas

## IDH

In [153]:
ihdMunicipio = pd.read_csv(prata_idh + '/IdhMunicipio.csv',sep=';')  

#Filtrando somente as colunas necessárias
ihdMunicipio = ihdMunicipio.filter(regex='re.|localidade')
ihdMunicipio = ihdMunicipio.dropna(axis=1, how='all')

#Criação do df com a seguinte estrutura: IdMunicipio, Ano e Valor
df_ihdMunicipio = pd.DataFrame()
for index, row in ihdMunicipio.iterrows():
     
    # Lendo as colunas da linha
    for col in row.index:
         
         # Seleciondo os anos
        if len(re.findall(r'\d+', col)) > 0:
            row_new = {'IdMunicipio':[int(row['localidade'])],'Ano': re.findall(r'\d+', col) ,'Valor': [row[col]]}
            row_new = pd.DataFrame(row_new) 
            df_ihdMunicipio = pd.concat([row_new,df_ihdMunicipio]) 

#Criando a PK
df_ihdMunicipio["PkIdhMunicipio"] = df_ihdMunicipio['IdMunicipio'].astype(str) + df_ihdMunicipio["Ano"]

df_ihdMunicipio = df_ihdMunicipio.rename(columns={'Valor': "ValorIhdMunicipio"})

# Tipando a coluna númerico
df_ihdMunicipio['ValorIhdMunicipio'] = df_ihdMunicipio['ValorIhdMunicipio'].apply(lambda x: ConverterNumero(x))

ihdMunicipio_ouro = diretorio_ouro + '/ihd_municipio.csv'

pd.merge(df_ihdMunicipio, municipio[['PkIdMunicipio','IdEstado']], \
         left_on=['IdMunicipio'], right_on=['PkIdMunicipio'], how='inner') \
        .to_csv(ihdMunicipio_ouro, encoding='utf-8', sep=';',index=False)

del df_ihdMunicipio, ihdMunicipio

In [154]:
ihdEstado = pd.read_csv(prata_idh + '/IdhEstado.csv',sep=';')  

#Filtrando somente as colunas necessárias
ihdEstado = ihdEstado.filter(regex='re.|localidade')
ihdEstado = ihdEstado.dropna(axis=1, how='all')

#Criação do df com a seguinte estrutura: IdMunicipio, Ano e Valor
df = pd.DataFrame()
for index, row in ihdEstado.iterrows():
    
     # Lendo as colunas da linha
     for col in row.index:
         
         # Seleciondo apenas os anos
        if len(re.findall(r'\d+', col)) > 0:
            row_new = {'IdEstado':[int(row['localidade'])],'Ano': re.findall(r'\d+', col) ,'Valor': [row[col]]}
            row_new = pd.DataFrame(row_new) 
            df = pd.concat([row_new,df]) 

#Criando a PK
df["PkIdEstado"] = df['IdEstado'].astype(str) + df["Ano"]

df = df.rename(columns={'Valor': "ValorIhdEstado"})

# Tipando a coluna númerico
df['ValorIhdEstado'] = df['ValorIhdEstado'].apply(lambda x: ConverterNumero(x))

ihdEstado = df
del df

ihdEstado_ouro = diretorio_ouro + '/ihd_estado.csv'
ihdEstado.to_csv(ihdEstado_ouro, encoding='utf-8', sep=';',index=False)

del ihdEstado

## PIB

In [155]:
pib = pd.read_csv(prata_pib + '/pib.csv',sep=';')  

#Filtrando somente os registros dos municípios
pib = pib[pib['NC'] == '6']

#Filtrando somente as colunas necessárias
pib = pib[['D1C','D3N','V']]
filter = pib['D1C'].str.contains('Código')
pib = pib[~filter]
pib = pib.rename(columns={'D1C': "IdMunicipio",'D3N':'Ano','V':'ValorPib'})

#Criando a PK
pib["PkPib"] = pib['IdMunicipio'].astype(str) + pib["Ano"]

# Tipando a coluna númerico
pib['ValorPib'] = pib['ValorPib'].apply(lambda x: ConverterNumero(x))
pib['IdMunicipio'] = pib['IdMunicipio'].astype('Int64')
pib['Ano'] = pib['Ano'].astype('Int64')

# Gerando a tabela da ouro
pib_bruto_ouro = diretorio_ouro + '/pib.csv'

pd.merge(pib, municipio[['PkIdMunicipio','IdEstado']],left_on=['IdMunicipio'], right_on=['PkIdMunicipio'], how='inner') \
    .to_csv(pib_bruto_ouro, encoding='utf-8', sep=';',index=False)

del pib, municipio, estado

## Fato IBGE

In [156]:
localidade = pd.read_csv(diretorio_ouro + '/localidade.csv',sep=';')  
populacao = pd.read_csv(diretorio_ouro + '/populacao.csv',sep=';')  
ihdEstado = pd.read_csv(diretorio_ouro + '/ihd_estado.csv',sep=';')  
ihdMunicipio = pd.read_csv(diretorio_ouro + '/ihd_municipio.csv',sep=';')  
pib = pd.read_csv(diretorio_ouro + '/pib.csv',sep=';')  

In [157]:
# Tabelas de suporte para a criação da Fato IBGE

fato_localidade = localidade[['PkIdMunicipio','IdEstado']]
fato_populacao = populacao[['PkPopulacao','IdMunicipio','Ano','IdSexo','IdRangeIdade','QuantidadeSexoRangeIdade','IdCorERaca','QuantidadeCorERaca']]
fato_pib = pib[['PkPib','IdMunicipio','IdEstado','Ano','ValorPib']]

In [158]:
# ------------------------ Merge entre a Localidade e População ------------------------
fatoIbge = pd.merge(fato_localidade, fato_populacao,left_on='PkIdMunicipio', right_on='IdMunicipio', how='inner').drop('IdMunicipio', axis=1)

# ------------------------ Merge com PIB ------------------------
fatoIbge = pd.merge(fatoIbge, fato_pib, left_on=['PkIdMunicipio','Ano'], right_on=['IdMunicipio','Ano'], how='outer')
               
# Atualizar a PkIdMunicipio
fatoIbge.loc[fatoIbge['PkIdMunicipio'].isna(), 'PkIdMunicipio'] = fatoIbge['IdMunicipio']

# Atualizar a IdEstado
fatoIbge.loc[fatoIbge['IdEstado_x'].isna(), 'IdEstado_x'] = fatoIbge['IdEstado_y']

fatoIbge = fatoIbge.rename(columns={'IdEstado_x': 'IdEstado'}).drop('IdEstado_y', axis=1)


# ------------------------ Merge com IHD Município (IHDM) ------------------------
fatoIbge = pd.merge(fatoIbge, ihdMunicipio, \
            left_on=['PkIdMunicipio','Ano'], right_on=['PkIdMunicipio','Ano'], how='outer')

# Atualizar a IdEstado
fatoIbge.loc[fatoIbge['IdEstado_x'].isna(), 'IdEstado_x'] = \
    fatoIbge[~fatoIbge['IdEstado_y'].isna()]['IdEstado_y']

fatoIbge = fatoIbge.rename(columns={'IdEstado_x':'IdEstado'}) \
                     .drop(['IdMunicipio_x','IdMunicipio_y','IdEstado_y'], axis=1)


# ------------------------ Merge com IHD Estado ------------------------
fatoIbge = pd.merge(fatoIbge, ihdEstado,left_on=['IdEstado','Ano'], right_on=['IdEstado','Ano'], how='outer')

# Atualizar os campos das quantidades e valores para 0 caso for nulo
fatoIbge['QuantidadeSexoRangeIdade'].fillna(0, inplace=True)
fatoIbge['ValorIhdMunicipio'].fillna(0, inplace=True)
fatoIbge['QuantidadeCorERaca'].fillna(0, inplace=True)
fatoIbge['ValorPib'].fillna(0, inplace=True)
fatoIbge['ValorIhdMunicipio'].fillna(0, inplace=True)
fatoIbge['ValorIhdEstado'].fillna(0, inplace=True)

# Criando a PK
fatoIbge['PkFatoIbge'] = fatoIbge.apply(lambda x: \
                            CreatePkFatoIbge(x['PkIdMunicipio'], x['Ano'],x['PkPopulacao'],x['PkPib'],x['PkIdhMunicipio']) \
                            ,axis=1)

fatoIbge.to_csv(diretorio_ouro + '/fato_ibge.csv', encoding='utf-8', sep=';',index=False)
del fatoIbge,pib,ihdMunicipio,ihdEstado,populacao,localidade,fato_localidade,fato_populacao,fato_pib

# Término Log

In [159]:
main_end = time.time()
print("Tempo do processamento da criação dos dados nas 3 camadas {}.".format(main_end - main_start))

Tempo do processamento da criação dos dados nas 3 camadas 23654.733254909515.
